### Data creation for ENSO events and their spatial regions

Compute ONI values, then extract corresponding tropical Pacific region SSTs for ENSO events. Save as netCDFs for future use. Save El Nino, La Nina, and corresponding experiment climatology (for computing ENSO event anomalies).

**Figure by: Maria J. Molina, NCAR**

In [1]:
import xarray as xr
import numpy as np
from climatico import enso
import matplotlib.pyplot as plt
import cftime
import cartopy
import cartopy.crs as ccrs
from cartopy.util import add_cyclic_point
from climatico.util import weighted_mean, pacific_lon
import matplotlib.patches as mpatches
import cartopy.feature as cfeature
from shapely.geometry.polygon import Polygon
from datetime import timedelta

In [2]:
from config import directory_figs, directory_data

In [3]:
def grab_enso_spatial(ds, indices, ds_oni, year1, year2, cutoff, filevar='SST', quarter=12):
    """
    Use the input seasonal file ('QS-DEC') and precomputed Nino-index to extract Nino(a) 
    event indices in time array of SST file.
    Then extract the variable associated with the Nino(a) events. These focus on DJF months.
    
    Args:
        ds (xarray data array): Seasonal variable to filter for plotting 
                                    (e.g., ds.resample(time='QS-DEC').mean(skipna=True)).
        indices (numpy array): ONI values as numpy array 
                                    (e.g., control_nino.resample(time='QS-DEC').mean(skipna=True).values).
        ds_oni (xarray data array): ONI seasonal xarray 
                                    (e.g., control_nino.resample(time='QS-DEC').mean(skipna=True)).
        year1 (int): First year for range filtering.
        year2 (int): Second year for range filtering.
        cutoff (float): Cutoff of ENSO events. E.g., +/-0.5 for ONI.
        filevar (str): Variable name for input ds. Defaults to ``SST.``
        quarter (int): Month of quarter start (e.g., 12, 3, 6, 9).
    """
    # do checks
    check1 = ds[filevar].isel(time=np.where(indices>=cutoff)[0])[(ds[filevar].isel(time=np.where(indices>=cutoff)[0])['time'].dt.month==quarter)].coords['time']
    check2 = ds_oni.isel(time=np.where(indices>=cutoff)[0])[(ds_oni.isel(time=np.where(indices>=cutoff)[0])['time'].dt.month==quarter)].coords['time']
    assert np.all(check1.values==check2.values), "Nino events don't match"
    print("Nino check passed")
    
    check1 = ds[filevar].isel(time=np.where(indices<=cutoff)[0])[(ds[filevar].isel(time=np.where(indices<=cutoff)[0])['time'].dt.month==quarter)].coords['time']
    check2 = ds_oni.isel(time=np.where(indices<=cutoff)[0])[(ds_oni.isel(time=np.where(indices<=cutoff)[0])['time'].dt.month==quarter)].coords['time']
    assert np.all(check1.values==check2.values), "Nina events don't match"
    print("Nina check passed")
    
    ### climo
    climatology = ds[filevar][(ds[filevar]['time'].dt.month==quarter)]
    climatology = climatology[(climatology['time'].dt.year>=year1)&(climatology['time'].dt.year<=year2)]
    climo = climatology.mean(dim='time')
    
    ### nino
    # filter for djf quarters
    sv_indices = ds[filevar].isel(time=np.where(indices>=cutoff)[0])[(ds[filevar].isel(time=np.where(indices>=cutoff)[0])['time'].dt.month==quarter)]
    # filter for correct year range
    sv_indices = sv_indices[(sv_indices['time'].dt.year>=year1)&(sv_indices['time'].dt.year<=year2)]
    # take spatial mean for plotting
    nino = sv_indices.mean(dim='time')
    print("Nino done")

    ### nina
    # filter for djf quarters
    sv_indices = ds[filevar].isel(time=np.where(indices<=cutoff)[0])[(ds[filevar].isel(time=np.where(indices<=cutoff)[0])['time'].dt.month==quarter)]
    # filter for correct year range
    sv_indices = sv_indices[(sv_indices['time'].dt.year>=year1)&(sv_indices['time'].dt.year<=year2)]
    # take spatial mean for plotting
    nina = sv_indices.mean(dim='time')
    print("Nina done")
    
    return nino, nina, climo

In [4]:
# list of filenames to do this for:
# SSTs
file_g02sv = 'b1d.e11.B1850LENS.f09_g16.FWAtSalG02Sv.pop.h.SST.*.nc'
file_g04sv = 'b1d.e11.B1850LENS.f09_g16.FWAtSalG04Sv.pop.h.SST.*.nc'
file_p02sv = 'b1d.e11.B1850LENS.f09_g16.FWAtSalP02Sv.pop.h.SST.*.nc'
file_p04sv = 'b1d.e11.B1850LENS.f09_g16.FWAtSalP04Sv.pop.h.SST.*.nc'
file_psalt = 'b1d.e11.B1850LENS.f09_g16.FWPaSalP04Sv.pop.h.SST.*.nc'
file_cntrl = 'b1d.e11.B1850C5CN.f09_g16.005.pop.h.SST.*.nc'
# wind stress x-direction
taux_g02sv = 'b.e11.B1850LENS.f09_g16.FWAtSalG02Sv.cam.h0.TAUX.*.nc'
taux_g04sv = 'b.e11.B1850LENS.f09_g16.FWAtSalG04Sv.cam.h0.TAUX.*.nc'
taux_p02sv = 'b.e11.B1850LENS.f09_g16.FWAtSalP02Sv.cam.h0.TAUX.*.nc'
taux_p04sv = 'b.e11.B1850LENS.f09_g16.FWAtSalP04Sv.cam.h0.TAUX.*.nc'
taux_psalt = 'b.e11.B1850LENS.f09_g16.FWPaSalP04Sv.cam.h0.TAUX.*.nc'
taux_cntrl = 'b.e11.B1850C5CN.f09_g16.005.cam.h0.TAUX.*.nc'
# wind stress y-direction
tauy_g02sv = 'b.e11.B1850LENS.f09_g16.FWAtSalG02Sv.cam.h0.TAUY.*.nc'
tauy_g04sv = 'b.e11.B1850LENS.f09_g16.FWAtSalG04Sv.cam.h0.TAUY.*.nc'
tauy_p02sv = 'b.e11.B1850LENS.f09_g16.FWAtSalP02Sv.cam.h0.TAUY.*.nc'
tauy_p04sv = 'b.e11.B1850LENS.f09_g16.FWAtSalP04Sv.cam.h0.TAUY.*.nc'
tauy_psalt = 'b.e11.B1850LENS.f09_g16.FWPaSalP04Sv.cam.h0.TAUY.*.nc'
tauy_cntrl = 'b.e11.B1850C5CN.f09_g16.005.cam.h0.TAUY.*.nc'

In [5]:
nino = enso.DefineNino(nino='nino34', lats='lat', lons='lon', cutoff=0.5, runningmean=3)
ninoslab = enso.DefineNino(nino='pacslab', lats='lat', lons='lon', cutoff=0.5, runningmean=3)

In [6]:
####################################################################

ds2 = xr.open_mfdataset(f'{directory_data}{file_cntrl}',
                        combine='by_coords',
                        preprocess=nino.nino)
# fix time coord
ds2 = ds2.assign_coords(time=ds2.coords['time'] - timedelta(days=17))
# reduce dims to time, lat, lon
ds2 = ds2.isel(z_t=0).sel(time=slice(cftime.DatetimeNoLeap(800, 1, 1, 0, 0),cftime.DatetimeNoLeap(1600, 1, 1, 0, 0)))

print("indices started")
# rolling climo 
control_ssts_roll = nino.monthly_climo(ds2['SST'].chunk({'time':900}), yrsroll=30, centered=True, time='time')
# compute nino index
control_nino = nino.compute_index(ds2['SST'], control_ssts_roll, 
                                  linear_detrend=False, lat_name='lat')
# compute indices (previously)
#control_nino = nino.compute_index(ds2['SST'].groupby('time.month'), 
#                                  ds2['SST'].groupby('time.month').mean(skipna=True), 
#                                  linear_detrend=False, lat_name='lat')
# grab numpy array
control_index = control_nino.resample(time='QS-DEC').mean(skipna=True).values
print("indices done")

#################################################################### sst

dsslab = xr.open_mfdataset(f'{directory_data}{file_cntrl}',
                            combine='by_coords',
                            preprocess=ninoslab.nino)
# fix time coord
dsslab = dsslab.assign_coords(time=dsslab.coords['time'] - timedelta(days=17))
# reduce dims to time, lat, lon
dsslab = dsslab.isel(z_t=0).sel(time=slice(cftime.DatetimeNoLeap(800, 1, 1, 0, 0),cftime.DatetimeNoLeap(1600, 1, 1, 0, 0)))

# ssts
controlnino, controlnina, controlclimo = grab_enso_spatial(
                                              ds = dsslab.resample(time='QS-DEC').mean(skipna=True), 
                                              indices = control_index, 
                                              ds_oni = control_nino.resample(time='QS-DEC').mean(skipna=True),
                                              year1 = 801, 
                                              year2 = 1599,
                                              cutoff = nino.cutoff, 
                                              filevar = 'SST')

#################################################################### taux

dstx = xr.open_mfdataset(f'{directory_data}{taux_cntrl}',
                         combine='by_coords',
                         preprocess=ninoslab.nino)
# fix time coord
dstx = dstx.assign_coords(time=dstx.coords['time'] - timedelta(days=17))
# reduce dims to time, lat, lon
dstx = dstx.sel(time=slice(cftime.DatetimeNoLeap(800, 1, 1, 0, 0),cftime.DatetimeNoLeap(1600, 1, 1, 0, 0)))

# grab enso mean spatial maps
controlninotx, controlninatx, controlclimotx = grab_enso_spatial(
                                              ds = dstx.resample(time='QS-DEC').mean(skipna=True), 
                                              indices = control_index, 
                                              ds_oni = control_nino.resample(time='QS-DEC').mean(skipna=True),
                                              year1 = 801, 
                                              year2 = 1599,
                                              cutoff = nino.cutoff, 
                                              filevar = 'TAUX')

#################################################################### tauy

dsty = xr.open_mfdataset(f'{directory_data}{tauy_cntrl}',
                         combine='by_coords',
                         preprocess=ninoslab.nino)
# fix time coord
dsty = dsty.assign_coords(time=dsty.coords['time'] - timedelta(days=17))
# reduce dims to time, lat, lon
dsty = dsty.sel(time=slice(cftime.DatetimeNoLeap(800, 1, 1, 0, 0),cftime.DatetimeNoLeap(1600, 1, 1, 0, 0)))

# grab enso mean spatial maps
controlninoty, controlninaty, controlclimoty = grab_enso_spatial(
                                              ds = dsty.resample(time='QS-DEC').mean(skipna=True), 
                                              indices = control_index, 
                                              ds_oni = control_nino.resample(time='QS-DEC').mean(skipna=True),
                                              year1 = 801, 
                                              year2 = 1599, 
                                              cutoff = nino.cutoff, 
                                              filevar = 'TAUY')

####################################################################

indices started
indices done
Nino check passed
Nina check passed
Nino done
Nina done
Nino check passed
Nina check passed
Nino done
Nina done
Nino check passed
Nina check passed
Nino done
Nina done


In [7]:
####################################################################

ds1 = xr.open_mfdataset(f'{directory_data}{file_g02sv}',
                        combine='by_coords',
                        preprocess=nino.nino)
# fix time coord
ds1 = ds1.assign_coords(time=ds1.coords['time'] - timedelta(days=17))
# reduce dims to time, lat, lon
ds1 = ds1.isel(z_t=0)

print("indices started")
# rolling climo 
g02sv_ssts_roll = nino.monthly_climo(ds1['SST'].chunk({'time':900}), yrsroll=30, centered=True, time='time')
# compute nino index
g02sv_nino = nino.compute_index(ds1['SST'], g02sv_ssts_roll, 
                                linear_detrend=False, lat_name='lat')
# grab numpy array
g02sv_index = g02sv_nino.resample(time='QS-DEC').mean(skipna=True).values
print("indices done")

#################################################################### ssts

dsslab = xr.open_mfdataset(f'{directory_data}{file_g02sv}',
                            combine='by_coords',
                            preprocess=ninoslab.nino)
# fix time coord
dsslab = dsslab.assign_coords(time=dsslab.coords['time'] - timedelta(days=17))
# reduce dims to time, lat, lon
dsslab = dsslab.isel(z_t=0)

g02svnino, g02svnina, g02svclimo = grab_enso_spatial(
                                              ds = dsslab.resample(time='QS-DEC').mean(skipna=True), 
                                              indices = g02sv_index, 
                                              ds_oni = g02sv_nino.resample(time='QS-DEC').mean(skipna=True),
                                              year1 = 201, 
                                              year2 = 500,
                                              cutoff = nino.cutoff, 
                                              filevar = 'SST')

#################################################################### taux

dstx = xr.open_mfdataset(f'{directory_data}{taux_g02sv}',
                         combine='by_coords',
                         preprocess=ninoslab.nino)
# fix time coord
dstx = dstx.assign_coords(time=dstx.coords['time'] - timedelta(days=17))

# grab enso mean spatial maps
g02svninotx, g02svninatx, g02svclimotx = grab_enso_spatial(
                                              ds = dstx.resample(time='QS-DEC').mean(skipna=True), 
                                              indices = g02sv_index,
                                              ds_oni = g02sv_nino.resample(time='QS-DEC').mean(skipna=True),
                                              year1 = 201, 
                                              year2 = 500, 
                                              cutoff = nino.cutoff, 
                                              filevar = 'TAUX')


#################################################################### tauy

dsty = xr.open_mfdataset(f'{directory_data}{tauy_g02sv}',
                         combine='by_coords',
                         preprocess=ninoslab.nino)
# fix time coord
dsty = dsty.assign_coords(time=dsty.coords['time'] - timedelta(days=17))

# grab enso mean spatial maps
g02svninoty, g02svninaty, g02svclimoty = grab_enso_spatial(
                                              ds = dsty.resample(time='QS-DEC').mean(skipna=True), 
                                              indices = g02sv_index, 
                                              ds_oni = g02sv_nino.resample(time='QS-DEC').mean(skipna=True),
                                              year1 = 201, 
                                              year2 = 500, 
                                              cutoff = nino.cutoff, 
                                              filevar = 'TAUY')

####################################################################

indices started
indices done
Nino check passed
Nina check passed
Nino done
Nina done
Nino check passed
Nina check passed
Nino done
Nina done
Nino check passed
Nina check passed
Nino done
Nina done


In [8]:
####################################################################

ds1 = xr.open_mfdataset(f'{directory_data}{file_g04sv}',
                        combine='by_coords',
                        preprocess=nino.nino)
# fix time coord
ds1 = ds1.assign_coords(time=ds1.coords['time'] - timedelta(days=17))
# reduce dims to time, lat, lon
ds1 = ds1.isel(z_t=0)

print("indices started")
# rolling climo 
g04sv_ssts_roll = nino.monthly_climo(ds1['SST'].chunk({'time':900}), yrsroll=30, centered=True, time='time')
# compute nino index
g04sv_nino = nino.compute_index(ds1['SST'], g04sv_ssts_roll,
                                linear_detrend=False, lat_name='lat')
# grab numpy array
g04sv_index = g04sv_nino.resample(time='QS-DEC').mean(skipna=True).values
print("indices done")

#################################################################### ssts

dsslab = xr.open_mfdataset(f'{directory_data}{file_g04sv}',
                            combine='by_coords',
                            preprocess=ninoslab.nino)
# fix time coord
dsslab = dsslab.assign_coords(time=dsslab.coords['time'] - timedelta(days=17))
# reduce dims to time, lat, lon
dsslab = dsslab.isel(z_t=0)

g04svnino, g04svnina, g04svclimo = grab_enso_spatial(
                                              ds = dsslab.resample(time='QS-DEC').mean(skipna=True), 
                                              indices = g04sv_index, 
                                              ds_oni = g04sv_nino.resample(time='QS-DEC').mean(skipna=True),
                                              year1 = 201, 
                                              year2 = 500,
                                              cutoff = nino.cutoff, 
                                              filevar = 'SST')

#################################################################### taux

dstx = xr.open_mfdataset(f'{directory_data}{taux_g04sv}',
                         combine='by_coords',
                         preprocess=ninoslab.nino)
# fix time coord
dstx = dstx.assign_coords(time=dstx.coords['time'] - timedelta(days=17))

# grab enso mean spatial maps
g04svninotx, g04svninatx, g04svclimotx = grab_enso_spatial(
                                              ds = dstx.resample(time='QS-DEC').mean(skipna=True), 
                                              indices = g04sv_index, 
                                              ds_oni = g04sv_nino.resample(time='QS-DEC').mean(skipna=True),
                                              year1 = 201, 
                                              year2 = 500, 
                                              cutoff = nino.cutoff, 
                                              filevar = 'TAUX')

#################################################################### tauy

dsty = xr.open_mfdataset(f'{directory_data}{tauy_g04sv}',
                         combine='by_coords',
                         preprocess=ninoslab.nino)
# fix time coord
dsty = dsty.assign_coords(time=dsty.coords['time'] - timedelta(days=17))

# grab enso mean spatial maps
g04svninoty, g04svninaty, g04svclimoty = grab_enso_spatial(
                                              ds = dsty.resample(time='QS-DEC').mean(skipna=True), 
                                              indices = g04sv_index, 
                                              ds_oni = g04sv_nino.resample(time='QS-DEC').mean(skipna=True),
                                              year1 = 201, 
                                              year2 = 500, 
                                              cutoff = nino.cutoff, 
                                              filevar = 'TAUY')

####################################################################

indices started
indices done
Nino check passed
Nina check passed
Nino done
Nina done
Nino check passed
Nina check passed
Nino done
Nina done
Nino check passed
Nina check passed
Nino done
Nina done


In [9]:
####################################################################

ds1 = xr.open_mfdataset(f'{directory_data}{file_p02sv}',
                        combine='by_coords',
                        preprocess=nino.nino)
# fix time coord
ds1 = ds1.assign_coords(time=ds1.coords['time'] - timedelta(days=17))
# reduce dims to time, lat, lon
ds1 = ds1.isel(z_t=0)

print("indices started")
# rolling climo 
p02sv_ssts_roll = nino.monthly_climo(ds1['SST'].chunk({'time':900}), yrsroll=30, centered=True, time='time')
# compute nino index
p02sv_nino = nino.compute_index(ds1['SST'], p02sv_ssts_roll, 
                                linear_detrend=False, lat_name='lat')
# grab numpy array
p02sv_index = p02sv_nino.resample(time='QS-DEC').mean(skipna=True).values
print("indices done")

#################################################################### ssts

dsslab = xr.open_mfdataset(f'{directory_data}{file_p02sv}',
                            combine='by_coords',
                            preprocess=ninoslab.nino)
# fix time coord
dsslab = dsslab.assign_coords(time=dsslab.coords['time'] - timedelta(days=17))
# reduce dims to time, lat, lon
dsslab = dsslab.isel(z_t=0)

p02svnino, p02svnina, p02svclimo = grab_enso_spatial(
                                              ds = dsslab.resample(time='QS-DEC').mean(skipna=True), 
                                              indices = p02sv_index, 
                                              ds_oni = p02sv_nino.resample(time='QS-DEC').mean(skipna=True),
                                              year1 = 201, 
                                              year2 = 500, 
                                              cutoff = nino.cutoff, 
                                              filevar = 'SST')

#################################################################### taux

dstx = xr.open_mfdataset(f'{directory_data}{taux_p02sv}',
                         combine='by_coords',
                         preprocess=ninoslab.nino)
# fix time coord
dstx = dstx.assign_coords(time=dstx.coords['time'] - timedelta(days=17))

# grab enso mean spatial maps
p02svninotx, p02svninatx, p02svclimotx = grab_enso_spatial(
                                              ds = dstx.resample(time='QS-DEC').mean(skipna=True), 
                                              indices = p02sv_index, 
                                              ds_oni = p02sv_nino.resample(time='QS-DEC').mean(skipna=True),
                                              year1 = 201, 
                                              year2 = 500, 
                                              cutoff = nino.cutoff, 
                                              filevar = 'TAUX')

#################################################################### tauy

dsty = xr.open_mfdataset(f'{directory_data}{tauy_p02sv}',
                         combine='by_coords',
                         preprocess=ninoslab.nino)
# fix time coord
dsty = dsty.assign_coords(time=dsty.coords['time'] - timedelta(days=17))

# grab enso mean spatial maps
p02svninoty, p02svninaty, p02svclimoty = grab_enso_spatial(
                                              ds = dsty.resample(time='QS-DEC').mean(skipna=True), 
                                              indices = p02sv_index, 
                                              ds_oni = p02sv_nino.resample(time='QS-DEC').mean(skipna=True),
                                              year1 = 201, 
                                              year2 = 500, 
                                              cutoff = nino.cutoff, 
                                              filevar = 'TAUY')

####################################################################

indices started
indices done
Nino check passed
Nina check passed
Nino done
Nina done
Nino check passed
Nina check passed
Nino done
Nina done
Nino check passed
Nina check passed
Nino done
Nina done


In [10]:
####################################################################

ds1 = xr.open_mfdataset(f'{directory_data}{file_p04sv}',
                        combine='by_coords',
                        preprocess=nino.nino)
# fix time coord
ds1 = ds1.assign_coords(time=ds1.coords['time'] - timedelta(days=17))
# reduce dims to time, lat, lon
ds1 = ds1.isel(z_t=0)

print("indices started")
# rolling climo 
p04sv_ssts_roll = nino.monthly_climo(ds1['SST'].chunk({'time':900}), yrsroll=30, centered=True, time='time')
# compute nino index
p04sv_nino = nino.compute_index(ds1['SST'], p04sv_ssts_roll, 
                                linear_detrend=False, lat_name='lat')
# grab numpy array
p04sv_index = p04sv_nino.resample(time='QS-DEC').mean(skipna=True).values
print("indices done")

#################################################################### ssts

dsslab = xr.open_mfdataset(f'{directory_data}{file_p04sv}',
                            combine='by_coords',
                            preprocess=ninoslab.nino)
# fix time coord
dsslab = dsslab.assign_coords(time=dsslab.coords['time'] - timedelta(days=17))
# reduce dims to time, lat, lon
dsslab = dsslab.isel(z_t=0)

p04svnino, p04svnina, p04svclimo = grab_enso_spatial(
                                              ds = dsslab.resample(time='QS-DEC').mean(skipna=True), 
                                              indices = p04sv_index, 
                                              ds_oni = p04sv_nino.resample(time='QS-DEC').mean(skipna=True),
                                              year1 = 201, 
                                              year2 = 500, 
                                              cutoff = nino.cutoff, 
                                              filevar = 'SST')

#################################################################### taux

dstx = xr.open_mfdataset(f'{directory_data}{taux_p04sv}',
                         combine='by_coords',
                         preprocess=ninoslab.nino)
# fix time coord
dstx = dstx.assign_coords(time=dstx.coords['time'] - timedelta(days=17))

# grab enso mean spatial maps
p04svninotx, p04svninatx, p04svclimotx = grab_enso_spatial(
                                              ds = dstx.resample(time='QS-DEC').mean(skipna=True), 
                                              indices = p04sv_index, 
                                              ds_oni = p04sv_nino.resample(time='QS-DEC').mean(skipna=True),
                                              year1 = 201, 
                                              year2 = 500, 
                                              cutoff = nino.cutoff, 
                                              filevar='TAUX')

#################################################################### tauy

dsty = xr.open_mfdataset(f'{directory_data}{tauy_p04sv}',
                         combine='by_coords',
                         preprocess=ninoslab.nino)
# fix time coord
dsty = dsty.assign_coords(time=dsty.coords['time'] - timedelta(days=17))

# grab enso mean spatial maps
p04svninoty, p04svninaty, p04svclimoty = grab_enso_spatial(
                                              ds = dsty.resample(time='QS-DEC').mean(skipna=True), 
                                              indices = p04sv_index, 
                                              ds_oni = p04sv_nino.resample(time='QS-DEC').mean(skipna=True),
                                              year1 = 201, 
                                              year2 = 500, 
                                              cutoff = nino.cutoff, 
                                              filevar = 'TAUY')

####################################################################

indices started
indices done
Nino check passed
Nina check passed
Nino done
Nina done
Nino check passed
Nina check passed
Nino done
Nina done
Nino check passed
Nina check passed
Nino done
Nina done


In [11]:
####################################################################

ds1 = xr.open_mfdataset(f'{directory_data}{file_psalt}',
                        combine='by_coords',
                        preprocess=nino.nino)
# fix time coord
ds1 = ds1.assign_coords(time=ds1.coords['time'] - timedelta(days=17))
# reduce dims to time, lat, lon
ds1 = ds1.isel(z_t=0)

# psalt index computation
ds8 = xr.open_mfdataset(f'{directory_data}b2d.e11.B1850LENS.f09_g16.FWPaSalP04Sv.pop.h.SST.030101-035012.nc',
                       combine='by_coords',
                       preprocess=nino.nino)
# fix time coord
ds8 = ds8.assign_coords(time=ds8.coords['time'] - timedelta(days=17))
ds8 = ds8.sel(time=slice('0301-01-01 00:00:00', '0351-01-01 00:00:00'))

# attach first 100 years
pnew = xr.concat([ds1['SST'].sel(time=slice('0001-01-01 00:00:00', '0301-01-01 00:00:00')).drop('z_t'),
                  ds8['SST']], dim='time')

print("indices started")
# rolling climo 
psalt_ssts_roll = nino.monthly_climo(pnew.chunk({'time':900}), yrsroll=30, centered=True, time='time')
# compute nino index
psalt_nino = nino.compute_index(pnew, psalt_ssts_roll, 
                                linear_detrend=False, lat_name='lat')
psalt_nino = psalt_nino.sel(time=slice('0001-01-01 00:00:00', '0301-01-01 00:00:00'))
# grab numpy array
psalt_index = psalt_nino.resample(time='QS-DEC').mean(skipna=True).values
print("indices done")

#################################################################### ssts

dsslab = xr.open_mfdataset(f'{directory_data}{file_psalt}',
                            combine='by_coords',
                            preprocess=ninoslab.nino)
# fix time coord
dsslab = dsslab.assign_coords(time=dsslab.coords['time'] - timedelta(days=17))
# reduce dims to time, lat, lon
dsslab = dsslab.isel(z_t=0)

psaltnino, psaltnina, psaltclimo = grab_enso_spatial(
                                              ds = dsslab.resample(time='QS-DEC').mean(skipna=True), 
                                              indices = psalt_index, 
                                              ds_oni = psalt_nino.resample(time='QS-DEC').mean(skipna=True),
                                              year1 = 101, 
                                              year2 = 250, 
                                              cutoff = nino.cutoff, 
                                              filevar = 'SST')

#################################################################### taux

dstx = xr.open_mfdataset(f'{directory_data}{taux_psalt}',
                         combine='by_coords',
                         preprocess=ninoslab.nino)
# fix time coord
dstx = dstx.assign_coords(time=dstx.coords['time'] - timedelta(days=17))

# grab enso mean spatial maps
psaltninotx, psaltninatx, psaltclimotx = grab_enso_spatial(
                                              ds = dstx.resample(time='QS-DEC').mean(skipna=True), 
                                              indices = psalt_index, 
                                              ds_oni = psalt_nino.resample(time='QS-DEC').mean(skipna=True),
                                              year1 = 101, 
                                              year2 = 250, 
                                              cutoff = nino.cutoff, 
                                              filevar = 'TAUX')

#################################################################### tauy

dsty = xr.open_mfdataset(f'{directory_data}{tauy_psalt}',
                         combine='by_coords',
                         preprocess=ninoslab.nino)
# fix time coord
dsty = dsty.assign_coords(time=dsty.coords['time'] - timedelta(days=17))

# grab enso mean spatial maps
psaltninoty, psaltninaty, psaltclimoty = grab_enso_spatial(
                                              ds = dsty.resample(time='QS-DEC').mean(skipna=True), 
                                              indices = psalt_index, 
                                              ds_oni = psalt_nino.resample(time='QS-DEC').mean(skipna=True),
                                              year1 = 101, 
                                              year2 = 250,
                                              cutoff = nino.cutoff, 
                                              filevar = 'TAUY')

####################################################################

indices started
indices done
Nino check passed
Nina check passed
Nino done
Nina done
Nino check passed
Nina check passed
Nino done
Nina done
Nino check passed
Nina check passed
Nino done
Nina done


In [12]:
%%capture
cntrl_diff_nino = controlnino.values
cntrl_diff_ninotx = np.negative(controlninotx).values
cntrl_diff_ninoty = np.negative(controlninoty).values

In [13]:
%%capture
g02sv_diff_nino = g02svnino.values
g02sv_diff_ninotx = np.negative(g02svninotx).values
g02sv_diff_ninoty = np.negative(g02svninoty).values

In [14]:
%%capture
g04sv_diff_nino = g04svnino.values
g04sv_diff_ninotx = np.negative(g04svninotx).values
g04sv_diff_ninoty = np.negative(g04svninoty).values

In [15]:
%%capture
p02sv_diff_nino = p02svnino.values
p02sv_diff_ninotx = np.negative(p02svninotx).values
p02sv_diff_ninoty = np.negative(p02svninoty).values

In [16]:
%%capture
p04sv_diff_nino = p04svnino.values
p04sv_diff_ninotx = np.negative(p04svninotx).values
p04sv_diff_ninoty = np.negative(p04svninoty).values

In [17]:
%%capture
psalt_diff_nino = psaltnino.values
psalt_diff_ninotx = np.negative(psaltninotx).values
psalt_diff_ninoty = np.negative(psaltninoty).values

In [18]:
data_assemble=xr.Dataset({
    
                 'cntrl_diff_nino':(['ssty','sstx'], cntrl_diff_nino),
                 'cntrl_diff_ninotx':(['y','x'], cntrl_diff_ninotx),
                 'cntrl_diff_ninoty':(['y','x'], cntrl_diff_ninoty),
    
                 'g02sv_diff_nino':(['ssty','sstx'], g02sv_diff_nino),
                 'g02sv_diff_ninotx':(['y','x'], g02sv_diff_ninotx),
                 'g02sv_diff_ninoty':(['y','x'], g02sv_diff_ninoty),
    
                 'g04sv_diff_nino':(['ssty','sstx'], g04sv_diff_nino),
                 'g04sv_diff_ninotx':(['y','x'], g04sv_diff_ninotx),
                 'g04sv_diff_ninoty':(['y','x'], g04sv_diff_ninoty),
    
                 'p02sv_diff_nino':(['ssty','sstx'], p02sv_diff_nino),
                 'p02sv_diff_ninotx':(['y','x'], p02sv_diff_ninotx),
                 'p02sv_diff_ninoty':(['y','x'], p02sv_diff_ninoty),
    
                 'p04sv_diff_nino':(['ssty','sstx'], p04sv_diff_nino),
                 'p04sv_diff_ninotx':(['y','x'], p04sv_diff_ninotx),
                 'p04sv_diff_ninoty':(['y','x'], p04sv_diff_ninoty),
    
                 'psalt_diff_nino':(['ssty','sstx'], psalt_diff_nino),
                 'psalt_diff_ninotx':(['y','x'], psalt_diff_ninotx),
                 'psalt_diff_ninoty':(['y','x'], psalt_diff_ninoty),
    
                },
                 coords=
                {'lon':(['sstx'], controlnino.coords['lon'].values),
                 'lat':(['ssty'], controlnino.coords['lat'].values),
                 'lons':(['x'], controlninotx.coords['lon'].values),
                 'lats':(['y'], controlninotx.coords['lat'].values)
                })     

In [19]:
data_assemble.to_netcdf(f'{directory_data}fig10_ninodata.nc')

In [20]:
%%capture
cntrl_diff_nina = controlnina.values
cntrl_diff_ninatx = np.negative(controlninatx).values
cntrl_diff_ninaty = np.negative(controlninaty).values

In [21]:
%%capture
g02sv_diff_nina = g02svnina.values
g02sv_diff_ninatx = np.negative(g02svninatx).values
g02sv_diff_ninaty = np.negative(g02svninaty).values

In [22]:
%%capture
g04sv_diff_nina = g04svnina.values
g04sv_diff_ninatx = np.negative(g04svninatx).values
g04sv_diff_ninaty = np.negative(g04svninaty).values

In [23]:
%%capture
p02sv_diff_nina = p02svnina.values
p02sv_diff_ninatx = np.negative(p02svninatx).values
p02sv_diff_ninaty = np.negative(p02svninaty).values

In [24]:
%%capture
p04sv_diff_nina = p04svnina.values
p04sv_diff_ninatx = np.negative(p04svninatx).values
p04sv_diff_ninaty = np.negative(p04svninaty).values

In [25]:
%%capture
psalt_diff_nina = psaltnina.values
psalt_diff_ninatx = np.negative(psaltninatx).values
psalt_diff_ninaty = np.negative(psaltninaty).values

In [26]:
data_assemble=xr.Dataset({
    
                 'cntrl_diff_nina':(['ssty','sstx'], cntrl_diff_nina),
                 'cntrl_diff_ninatx':(['y','x'], cntrl_diff_ninatx),
                 'cntrl_diff_ninaty':(['y','x'], cntrl_diff_ninaty),
    
                 'g02sv_diff_nina':(['ssty','sstx'], g02sv_diff_nina),
                 'g02sv_diff_ninatx':(['y','x'], g02sv_diff_ninatx),
                 'g02sv_diff_ninaty':(['y','x'], g02sv_diff_ninaty),
    
                 'g04sv_diff_nina':(['ssty','sstx'], g04sv_diff_nina),
                 'g04sv_diff_ninatx':(['y','x'], g04sv_diff_ninatx),
                 'g04sv_diff_ninaty':(['y','x'], g04sv_diff_ninaty),
    
                 'p02sv_diff_nina':(['ssty','sstx'], p02sv_diff_nina),
                 'p02sv_diff_ninatx':(['y','x'], p02sv_diff_ninatx),
                 'p02sv_diff_ninaty':(['y','x'], p02sv_diff_ninaty),
    
                 'p04sv_diff_nina':(['ssty','sstx'], p04sv_diff_nina),
                 'p04sv_diff_ninatx':(['y','x'], p04sv_diff_ninatx),
                 'p04sv_diff_ninaty':(['y','x'], p04sv_diff_ninaty),
    
                 'psalt_diff_nina':(['ssty','sstx'], psalt_diff_nina),
                 'psalt_diff_ninatx':(['y','x'], psalt_diff_ninatx),
                 'psalt_diff_ninaty':(['y','x'], psalt_diff_ninaty),
    
                },
                 coords=
                {'lon':(['sstx'], controlnina.coords['lon'].values),
                 'lat':(['ssty'], controlnina.coords['lat'].values),
                 'lons':(['x'], controlninatx.coords['lon'].values),
                 'lats':(['y'], controlninatx.coords['lat'].values)
                })

In [27]:
data_assemble.to_netcdf(f'{directory_data}fig11_ninadata.nc')

In [28]:
%%capture
cntrl_diff_climo = controlclimo.values
cntrl_diff_climotx = np.negative(controlclimotx).values
cntrl_diff_climoty = np.negative(controlclimoty).values

In [29]:
%%capture
g02sv_diff_climo = g02svclimo.values
g02sv_diff_climotx = np.negative(g02svclimotx).values
g02sv_diff_climoty = np.negative(g02svclimoty).values

In [30]:
%%capture
g04sv_diff_climo = g04svclimo.values
g04sv_diff_climotx = np.negative(g04svclimotx).values
g04sv_diff_climoty = np.negative(g04svclimoty).values

In [31]:
%%capture
p02sv_diff_climo = p02svclimo.values
p02sv_diff_climotx = np.negative(p02svclimotx).values
p02sv_diff_climoty = np.negative(p02svclimoty).values

In [32]:
%%capture
p04sv_diff_climo = p04svclimo.values
p04sv_diff_climotx = np.negative(p04svclimotx).values
p04sv_diff_climoty = np.negative(p04svclimoty).values

In [33]:
%%capture
psalt_diff_climo = psaltclimo.values
psalt_diff_climotx = np.negative(psaltclimotx).values
psalt_diff_climoty = np.negative(psaltclimoty).values

In [34]:
data_assemble=xr.Dataset({
    
                 'cntrl_diff_climo':(['ssty','sstx'], cntrl_diff_climo),
                 'cntrl_diff_climotx':(['y','x'], cntrl_diff_climotx),
                 'cntrl_diff_climoty':(['y','x'], cntrl_diff_climoty),
    
                 'g02sv_diff_climo':(['ssty','sstx'], g02sv_diff_climo),
                 'g02sv_diff_climotx':(['y','x'], g02sv_diff_climotx),
                 'g02sv_diff_climoty':(['y','x'], g02sv_diff_climoty),
    
                 'g04sv_diff_climo':(['ssty','sstx'], g04sv_diff_climo),
                 'g04sv_diff_climotx':(['y','x'], g04sv_diff_climotx),
                 'g04sv_diff_climoty':(['y','x'], g04sv_diff_climoty),
    
                 'p02sv_diff_climo':(['ssty','sstx'], p02sv_diff_climo),
                 'p02sv_diff_climotx':(['y','x'], p02sv_diff_climotx),
                 'p02sv_diff_climoty':(['y','x'], p02sv_diff_climoty),
    
                 'p04sv_diff_climo':(['ssty','sstx'], p04sv_diff_climo),
                 'p04sv_diff_climotx':(['y','x'], p04sv_diff_climotx),
                 'p04sv_diff_climoty':(['y','x'], p04sv_diff_climoty),
    
                 'psalt_diff_climo':(['ssty','sstx'], psalt_diff_climo),
                 'psalt_diff_climotx':(['y','x'], psalt_diff_climotx),
                 'psalt_diff_climoty':(['y','x'], psalt_diff_climoty),
    
                },
                 coords=
                {'lon':(['sstx'], controlclimo.coords['lon'].values),
                 'lat':(['ssty'], controlclimo.coords['lat'].values),
                 'lons':(['x'], controlclimotx.coords['lon'].values),
                 'lats':(['y'], controlclimotx.coords['lat'].values)
                })     

In [35]:
data_assemble.to_netcdf(f'{directory_data}fig11_climodata.nc')